# LIVRIA : Filtrage colllaboratif - Partie II

   Ce jupyter notebook contient le code permettant la prédiction des livres susceptibles de plaire à l'utilisateur. Nous entraînons et mesurons les prédictions ici, mais vous retrouverez la mise en forme des résultats de la prédiction dans le notebook Livra_recommender_system.ipynb.

   Nous avons dans un premier temps utilisé notre base de données issue du questionnaire afin de mettre en place des modèles de prédiction de thèmes, modèles basés sur des techniques de filtrage collaboratif. On a pu mesurer la perfomance des modèles et les comparer entre eux en se basant sur un type de mesure d'erreur entre les valeurs des données du set de test et les prédicions des différents modèles.
   Maintenant, nous allons pouvoir utiliser la base de données Goodbooks-10k pour réaliser le même travail de filtrage collaboratif mais cet fois-ci la prédiction sera portée sur des livres. 
   
* Le set de données de Goodbooks-10 :
http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/

Je précise que dans la dernière partie du notebook dataVizualisation&Cleaning_GoodBooks10k.ipynb, avons nettoyé le set de donnée "**ratings.csv**" que nous avons enregistré sous le nom "**df_notes.csv**" (cf. dossier ./data). Ainsi, nous avons gardé les livres les plus notés et les utilisateurs les plus actifs - ceux qui ont attribués le plus de notes - afin de faciliter le filtrage collaboratif et de contourner le problème de la taille du set complet qui était trop gros pour être utilisé entièrement ici.

## Import des librairies

On commence par importer les librairies utilisées dans ce notebook.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

## Import des données

Vous pouvez retrouver l'ensemble des données utilisées dans le dossier './data'


In [2]:
# Lecture du fichier 'df_notes.csv'
df_notes = pd.read_csv('data/df_notes.csv', sep='\t')
# On supprime la colonne inutile :
del df_notes['Unnamed: 0']

In [3]:
# On montre un extrait des notes attribuées
df_notes.head(10)

,user_id,book_id,rating
0,143,258,4
1,143,26,4
2,143,301,3
3,143,18,3
4,143,27,4
5,143,21,4
6,143,2,5
7,143,23,4
8,143,24,5
9,143,255,5


In [ ]:
print('Les dimensions de df_notes sont de : ' + str(df_notes.shape))

Les dimensions de df_notes sont de : (721076, 3)


## Création des sets d'entraînement et de test

Tout comme nous l'avons fait pour les thèmes préalablement (Cf. Partie I du filtrage collaboratif), on sépare le dataset en deux sets distincts : un pour l'entraînement de notre modèle de prédiction et un pour tester ce modèle. On garde 25% des données pour le set de test.

In [ ]:
train_data, test_data = train_test_split(df_notes, test_size=0.25)

## Filtrage collaboratif

On se base toujours sur les deux mêmes modèles pour le filtrage collaboratif : le "**memory-based**" et le "**model-based**".

Pour commencer, on crée une matrice utilisateur-livre pour l'entraînement du modèle de prédiction.

In [ ]:
train_data_matrix = np.zeros((4970,4750))
train_data_matrix = pd.DataFrame(train_data_matrix,index=df_notes['user_id'].unique(), columns=df_notes['book_id'].unique())
for row in train_data.itertuples():
    train_data_matrix[row[1], row[2]] = row[3]
train_data_matrix

On crée ensuite une matrice utilisateur-thème pour tester le modèle.

In [ ]:
test_data_matrix = np.zeros((4970,4750))
for row in test_data.itertuples():
    test_data_matrix[row[0], row[2]] = row[3]
test_data_matrix

## Filtrage collaboratif avec la méthode Memory-Based 

L'idée sous-jacente derrière le modèle dit "**memory-based**" est de calculer et d'utiliser les **similarités** entre utilisateurs et/ou items -ici les thèmes- et d'utiliser ces facteurs comme des "poids"  permettant la prédiction d'un thème, d'une note attribuée à un livre, ou autre. 

Nous allons tester les deux types de filtrage collaboratif:

* Item-Item 
* Utilisateur-Item 

Nous utilisons le coefficient de similarité. Pour cela, nous importons la fonction "pairwise_distances" de Scikit-Learn. 

On calcule d'abord la similarité entre les utilisateurs.

In [ ]:
from sklearn.metrics import pairwise
user_similarity_theme = pairwise.cosine_similarity(train_data_theme_matrix)

In [ ]:
user_similarity_theme[:5, 0:5]

Calcul de la similarité entre les thèmes :

In [ ]:
item_similarity_theme = pairwise.cosine_similarity(train_data_theme_matrix.T)

In [ ]:
item_similarity_theme[:5, 0:5]

On définit une méthode pour réaliser les prédictions. 

In [ ]:
def predict(choices, similarity, kind='user'):
    
    sum_sim = np.array([np.abs(similarity).sum(axis=1)])
    sum_sim[sum_sim == 0] = 1    
    if kind == 'user':
        return similarity.dot(choices) / sum_sim.T
    elif kind == 'item':
        return choices.dot(similarity) / sum_sim

Cette méthode permet de prédire les thèmes susceptibles d'intéresser un utilisateur. Soit elle prend en considération les thèmes qui lui plaisent déjà, soit elle regarde les thèmes de prédilection d'autres utilisateurs ayant donné des réponses similaires.

In [ ]:
item_prediction_theme = predict(train_data_theme_matrix, item_similarity_theme, 'item')

In [ ]:
item_prediction_theme[0:5,0:3]

In [ ]:
user_prediction_theme = predict(train_data_theme_matrix, user_similarity_theme, 'user')
user_prediction_theme[0:5,0:3]

On mesure la performance du modèle avec le calcul de la RMSE (root-mean-square error), c'est-à-dire la racine carrée de l'erreur quadratique. Cette méthode compare les vraies réponses aux réponses prédites par notre modèle.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, true_value):
    prediction = prediction.flatten()
    true_value = true_value.flatten()
    return sqrt(mean_squared_error(prediction, true_value))

RMSE pour la prédiction basée sur la comparaison entre les utilisateurs. 

In [ ]:
user_CF_RMSE_theme = rmse(user_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les utilisateurs : ', user_CF_RMSE_theme)

In [ ]:
item_CF_RMSE_theme = rmse(item_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les thèmes : ', item_CF_RMSE_theme)

## Filtrage collaboratif avec la méthode Model-based

La même logique développée dans la partie précédente (cf. I.1 memory-based collaborative filtering) peut être utilisée dans la méthode dite "model-based" : les similarités entre utilisateurs et/ou items peuvent être calculées et associées à un *modèle*, et on peut ensuite utiliser ce modèle pour faire nos prédictions. 

Le filtrage collaboratif dit "model-based" repose sur la factorisation de matrice. 

Nous allons utiliser un algorithme "SVD-based" permettant de réduire les dimensions de notre set de données et de guarder les caractéristiques principales, c'est-à-dire déterminantes de nos prédictions.

On regarde la proportion d'absence de données dans notre matrice.

In [ ]:
sparsity_theme=round(1.0-len(dataTheme)/float(1279*14),3)
print('The sparsity level of dataTheme is ' +  str(sparsity_theme*100) + '%')

Decompose the train_data_theme_matrix using the SVD method.

In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_theme_matrix, k = 13)

Crée une matrice diagonale.

In [ ]:
s_diag_matrix=np.diag(s)

Compute the rating predictions from the decomposition values.

In [ ]:
X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)

Compute the model RMSE.

In [ ]:
for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    u, s, vt = svds(train_data_theme_matrix, k = k)
    s_diag_matrix=np.diag(s)
    X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD-based CF RMSE (k={}): {}'.format(k, str(rmse(X_pred_theme, test_data_theme_matrix))))

### PCA avec Scikit-Learn 

In [ ]:
# on importe la librairie permettant la réduction de dimension de notre set de données sur les thèmes
from sklearn.decomposition import PCA 

Ici, on détermine précisemment le nombre minimum de dimensions à garder pour préserver au moins 95% de la variance caractérisant notre set de données.

In [ ]:
# On détermine d, le nombre de dimensions après PCA
pca = PCA()
pca.fit(train_data_theme)
cumsum = np.cumsum(pca.explained_variance_ratio_*100)
d = np.argmax(cumsum >= 95)+1

# On trace la variance cumulative en fonction du nombre de dimensions
dim=np.arange(1,15)
plt.plot(dim, cumsum)
axes = plt.gca()
plt.axhline(y=95,color="red")
plt.axvline(x=13, color="black", linestyle='--')
axes.xaxis.set_ticks(range(15))
plt.axvspan(12, 13, facecolor='#2ca02c', alpha=0.3)
plt.title("Variance expliquée en fonction du nombre de dimensions")
plt.xlabel("Dimensions")
plt.ylabel("Variance cumulative expliquée (%)")
plt.gcf().set_size_inches(15, 10)
plt.show()

print ('\n nombre de dimensions du set après PCA : ' + str(d))

Cela veut dire que l'on ne peut enlever qu'une seule dimension à notre set de données si on souhaite garder assez d'informations pour la prédiction. Cependant, comme nous pouvons le voir sur le graphique, réduire le set à 12 dimensions ne nous ferait pas dépasser de beaucoup la limite de variance cumulative généralement fixée à 95% de celle du set initial.